In [1]:
from nn import *

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

In [4]:
X_train, y_train = train_df.iloc[:, 1:], train_df.iloc[:, 0]
X_test, y_test = test_df.iloc[:, 1:], test_df.iloc[:, 0]

In [5]:
from tools import normalize

In [6]:
X_test = (X_test - X_train.mean()) / X_train.std()
X_train = (X_train - X_train.mean()) / X_train.std()

In [7]:
y_map = {"B": 0, "M":1}
y_train = y_train.map(y_map).values[:, np.newaxis].astype(np.float64)
y_test = y_test.map(y_map).values[:, np.newaxis].astype(np.float64)

In [8]:
def build_model(activation):
    nn = NeuralNetwork(loss='crossentropy', verbose=False)
    nn.add(Dense(30, 64, activation=activation))
    nn.add(Dense(64, 64, activation=activation))
    nn.add(Dense(64, 1, activation='sigmoid'))
    return nn

In [9]:
from metrics import accuracy

In [10]:
# n_episodes = 5
# activationss = ['sigmoid', 'tanh', 'relu', 'leaky_relu', 'elu']
# for activation in activations:
#     scores = []
#     np.random.seed(42)
#     for ep in range(n_episodes):
#         nn = build_model(activation)
#         nn.fit(X_train.values, y_train)
#         acc_train = accuracy(y_train, nn.predict(X_train) > .5)
#         acc_test = accuracy(y_test, nn.predict(X_test) > .5)
#         scores.append([acc_train, acc_test])
#     scores = np.array(scores)
#     mean_train_acc, mean_test_acc = scores.mean(axis=0)
#     print(f"activation={activation} | mean_train_acc={mean_train_acc} | mean_test_acc={mean_test_acc}")

In [11]:
from layers import *

In [12]:
nn = NeuralNetwork('binary_crossentropy', verbose=True, verbose_step=100, learning_rate=0.05, debug=False)
nn.add(Dense(30, 30, activation='relu'))
nn.add(Dense(30, 1, activation='sigmoid'))

nn.fit(X_train.values, y_train, n_epochs=500, metrics=['acc'])


[100/500]: loss=0.31365 acc=0.95540 
[200/500]: loss=0.11234 acc=0.97887 
[300/500]: loss=0.07668 acc=0.98122 
[400/500]: loss=0.06262 acc=0.99061 
[500/500]: loss=0.05518 acc=0.99061 


In [13]:
nn.fit(X_train.iloc[0, :].values.reshape(1, -1), y_train[0, :])

In [14]:
y_train[0, :]

array([0.])

In [15]:
# from keras.layers import Dense
# from keras.models import Sequential

# nk = Sequential()
# nk.add(Dense(10, activation='relu'))
# nk.add(Dense(1, activation='sigmoid'))
# nk.compile('sgd', loss='binary_crossentropy')
# nk.fit(X_train.values, y_train, epochs=10)

In [16]:
-np.log(1 - 0.11)

0.11653381625595151